In [2]:
import re
import numpy as np
from gensim.models import Word2Vec

## Dataset
dataset = [
    "သူမသည် ဖျက်သိမ်း စာကို ဖတ်နေစဉ် မျက်ရည်များ စီးကျလို့ နေသည်။",
    "အခန်းသည် သူမ မရှိခြင်းကြောင့် တိတ်ဆိတ်မှုတို့ဖြင့် လွှမ်းမိုး စေခဲ့သည်။",
    "သူသည် အလင်းမရှိသော နေရာတွင် တစ်ယောက်တည်း ထိုင်ပြီး ဝမ်းနည်းမှု၏ အငွေ့အသက်ကို ခံစားနေသည်။",
    "သူမ၏ နှလုံးသားသည် မပြောလိုက်သော စကားများ၏ နာကျင်မှုကြောင့် ဝမ်းနည်းနေသည်။",
    "မိုးသည် သူမ၏ စိတ်ခံစားချက်ကို စာနာပြီး ဝမ်းနည်းစွာ ကောင်းကင်မှ မျက်ရည်စက်များ အဖြစ် ရွာချပေးလို့ နေသည်။",

    "သူ၏ အပြုံးက တစ်ခန်းလုံးကို ထွန်းလင်းစေပြီး သူမ၏ နှလုံးသားကို ပူနွေး စေခဲ့သည်။",
    "သူမနှင့် တွေ့ဆုံတိုင်း ခံစားချက်တို့သည် အိမ်မက် တစ်ခု အလား အလွန်ပင် သာယာလှပလို့ နေသည်။",
    "အချစ်၏ စွမ်းအားတို့သည် ချစ်သူ နှစ်ဦး ကြားရှိ အကွာအဝေးကို ဖျက်ဆီးပစ်နိုင်သည်။",
    "အချစ်သည် သက်တမ်းမဲ့ တန်ခိုး တစ်ခု ဖြစ်ပြီး နှလုံးသားများကို နူးညံ့စေသည်။",
    "အချစ်ကို သန့်ရှင်းစွာ ထိန်းသိမ်းနိုင်ပါက ဘဝ၏ အလှတရားကို ခံစားရမည်။",

    "သူ၏ လက်သီးကို ကျစ်ကျစ်ဆုတ်ထားပြီး ရန်ငြိုးကို ထိန်းထားရန် ကြိုးစားနေသည်။",
    "သူမသည် တံခါးကို ပိတ်ချပြီး ဆူညံစွာ အော်ဟစ်လို့ နေသည်။",
    "သူ၏ မျက်လုံးများတွင် ဒေါသ စိတ်တို့ဖြင့် မီးဟုန်းဟုန်း လောင်နေသည်။",
    "အငြင်းပွားမှုသည် မာနကြောင့် တိုး၍ တိုး၍ လာသည်။",
    "သူမ၏ အသံသည် စိတ်ဆိုးမှုကြောင့် တုန်လှုပ်နေပြီး စူးစိမ့် လှသည်။"
]
targets = ["Sadness", "Sadness", "Sadness", "Sadness", "Sadness", "Love", "Love", "Love", "Love", "Love", "Anger", "Anger", "Anger", "Anger", "Anger"]

## Convert the target labels to numerical values
target_mapping = {"Sadness": 0, "Love": 1, "Anger": 2}
encoded_lbls = np.array([target_mapping[target] for target in targets])
print("Encoded Labels:", encoded_lbls) 

Encoded Labels: [0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


In [4]:
## Preprocess dataset: Tokenization and cleaning
def preprocess_text(text):
    ## Lowercase the text
    text = text.lower()  
    ## Tokenize by splitting on spaces
    tokens = text.split()  
    return tokens

## Tokenize the dataset
tokenized_data = [preprocess_text(sentence) for sentence in dataset]
print(f"Tokenized Data : {tokenized_data}")

Tokenized Data : [['သူမသည်', 'ဖျက်သိမ်း', 'စာကို', 'ဖတ်နေစဉ်', 'မျက်ရည်များ', 'စီးကျလို့', 'နေသည်။'], ['အခန်းသည်', 'သူမ', 'မရှိခြင်းကြောင့်', 'တိတ်ဆိတ်မှုတို့ဖြင့်', 'လွှမ်းမိုး', 'စေခဲ့သည်။'], ['သူသည်', 'အလင်းမရှိသော', 'နေရာတွင်', 'တစ်ယောက်တည်း', 'ထိုင်ပြီး', 'ဝမ်းနည်းမှု၏', 'အငွေ့အသက်ကို', 'ခံစားနေသည်။'], ['သူမ၏', 'နှလုံးသားသည်', 'မပြောလိုက်သော', 'စကားများ၏', 'နာကျင်မှုကြောင့်', 'ဝမ်းနည်းနေသည်။'], ['မိုးသည်', 'သူမ၏', 'စိတ်ခံစားချက်ကို', 'စာနာပြီး', 'ဝမ်းနည်းစွာ', 'ကောင်းကင်မှ', 'မျက်ရည်စက်များ', 'အဖြစ်', 'ရွာချပေးလို့', 'နေသည်။'], ['သူ၏', 'အပြုံးက', 'တစ်ခန်းလုံးကို', 'ထွန်းလင်းစေပြီး', 'သူမ၏', 'နှလုံးသားကို', 'ပူနွေး', 'စေခဲ့သည်။'], ['သူမနှင့်', 'တွေ့ဆုံတိုင်း', 'ခံစားချက်တို့သည်', 'အိမ်မက်', 'တစ်ခု', 'အလား', 'အလွန်ပင်', 'သာယာလှပလို့', 'နေသည်။'], ['အချစ်၏', 'စွမ်းအားတို့သည်', 'ချစ်သူ', 'နှစ်ဦး', 'ကြားရှိ', 'အကွာအဝေးကို', 'ဖျက်ဆီးပစ်နိုင်သည်။'], ['အချစ်သည်', 'သက်တမ်းမဲ့', 'တန်ခိုး', 'တစ်ခု', 'ဖြစ်ပြီး', 'နှလုံးသားများကို', 'နူးညံ့စေသည်။'], ['အချစ်ကို', 'သန့်ရှင်းစွာ', 'ထိန်းသိမ်းနိုင်

In [5]:
## Train a Word2Vec model using CBOW method
model = Word2Vec(sentences=tokenized_data, epochs=20, vector_size=50, window=5, min_count=1, workers=4)

## Save the model
model.save("custom_word2vec_cbow.model")

In [15]:
## Example: Get the vector for the word 'love'
love_vector = model.wv['သူမ']
print("Vector for 'သူမ':", love_vector)

Vector for 'သူမ': [ 0.01122973  0.01102897  0.00355839  0.01145792 -0.01787541  0.01314405
  0.01854455 -0.00840381  0.00322935 -0.01044927  0.00211077  0.00539053
  0.01639539  0.00119038  0.00497775  0.00265067  0.01682173 -0.01140771
 -0.01256685 -0.00727824 -0.00452823  0.01011147 -0.01614231 -0.00560236
 -0.01632411  0.01024872 -0.00513171 -0.01808014  0.008075    0.01797099
 -0.00616842 -0.0116442   0.00603058 -0.00088451 -0.01994174  0.01686896
 -0.01457232 -0.00986892 -0.00537068 -0.01084377  0.00352784  0.01941303
  0.00920052  0.01616042 -0.00084803  0.00131581 -0.00541018 -0.01768684
  0.00687991  0.00418625]


In [16]:
## Inference : Load the Custom Word2Vec model
custom_model = Word2Vec.load("custom_word2vec_cbow.model")

## Get word vectors from the loaded model
word_vectors = custom_model.wv

In [17]:
## Function to convert a sentence into a vector by averaging word vectors
def sentence_to_vector(sentence, word_vectors, vector_size=50):
    tokens = preprocess_text(sentence)
    vectors = [word_vectors[word] for word in tokens if word in word_vectors]
    ## If no word in the sentence is in the model, return a zero vector
    if len(vectors) == 0:  
        return np.zeros(vector_size)
    ## Average of all word vectors
    return np.mean(vectors, axis=0)  

## Convert the entire dataset into sentence vectors
features = np.array([sentence_to_vector(sentence, word_vectors, vector_size=50) for sentence in dataset])

## Example: Check the sentence vector for the first sentence
print(f"Sentence vector of '{dataset[0]}' is \n", features[0])

Sentence vector of 'သူမသည် ဖျက်သိမ်း စာကို ဖတ်နေစဉ် မျက်ရည်များ စီးကျလို့ နေသည်။' is 
 [-0.00241282  0.00170629  0.00903143  0.00211975  0.00521885 -0.00210046
  0.00648094  0.00494189  0.00010504 -0.00146664  0.00258958 -0.00484096
 -0.00271352  0.00304194  0.00471854 -0.00160088 -0.00177629  0.00406357
  0.0019526  -0.00493767  0.00763357  0.00140597  0.00671392  0.00357861
  0.00260323  0.00327013  0.00717568 -0.00270227  0.001588   -0.00188919
 -0.00522674 -0.00199754 -0.00306842 -0.0075189   0.00100979  0.00888417
  0.00238751 -0.00511943  0.0001655  -0.00060914  0.00416263  0.00153778
  0.00216832 -0.00026962  0.00328994  0.0059008  -0.00631203  0.00084557
  0.00411108  0.00469168]


In [18]:
## Classifier 
def cosine_similarity(A, B): 
    dot_product = np.dot(A, B) 
    norm_A = np.linalg.norm(A) 
    norm_B = np.linalg.norm(B) 
    return dot_product / (norm_A * norm_B) 

similarity = cosine_similarity(features[1], features[2]) 
print("Cosine Similarity:", similarity) 

Cosine Similarity: -0.091738634


In [25]:
## Inference 
test = "အချစ်၏ စွမ်းအားတို့သည် ချစ်သူ နှစ်ဦး ကြားရှိ အကွာအဝေးကို ဖျက်ဆီးပစ်နိုင်သည်။"
test_vector = sentence_to_vector(test, word_vectors)
preds = [cosine_similarity(source, test_vector) for source in features] 
pred = encoded_lbls[np.argmax(preds)]
redicted_emotion = [emotion for emotion, idx in target_mapping.items() if idx == pred]
print("Prediction :", redicted_emotion[0]) 

Prediction : Love
